In [ ]:
#pip install opencv-python
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

In [ ]:
# Função que realiza a leitura da imagem e
# faz os tratamentos necessários
def preprocess_image(image_path):
    
    # Carrega a imagem a partir do caminho especificado
    imagem = tf.image.decode_image(tf.io.read_file(image_path))
    
    # Verifica se a imagem tem um canal alfa (RGBA) (controla a opacidade da cor)
    if imagem.shape[-1] == 4:
        # Remove o canal alfa, deixando apenas os canais de cor RGB
        imagem = imagem[...,:-1]
    
    # Converte a imagem para um tipo de dados de ponto flutuante
    imagem = tf.cast(imagem, tf.float32)
    
    # Adiciona uma dimensão extra à imagem para que possa ser usada como entrada para o modelo
    return tf.expand_dims(imagem, 0)  
    
def save_image(imagem, filename):
    imagem = tf.squeeze(imagem)
    # Verifica se a imagem é um objeto PIL (Pillow), se não for, converte-a para uma imagem do TensorFlow
    if not isinstance(imagem, Image.Image):
        # Garante que os valores de pixel estejam no intervalo [0, 255]
        imagem = tf.clip_by_value(imagem, 0, 255)
        # Converte a imagem para um objeto PIL (Pillow)
        imagem = Image.fromarray(tf.cast(imagem, tf.uint8).numpy())
    
    # Salva a imagem em um arquivo com o nome especificado
    imagem.save("%s.jpg" % filename)

# Define uma função para plotar uma imagem
def plot_image(imagem, title=""):
    imagem = tf.squeeze(imagem)

    # Converte a imagem em um array NumPy
    imagem = np.asarray(imagem)
    
    # Garante que os valores de pixel estejam no intervalo [0, 255]
    imagem = tf.clip_by_value(imagem, 0, 255)
    
    # Converte a imagem para um objeto PIL (Pillow)
    imagem = Image.fromarray(tf.cast(imagem, tf.uint8).numpy())
    
    # Plota a imagem e remove os eixos do gráfico
    plt.imshow(imagem)
    plt.axis("off")
    
    # Define um título para o gráfico, se especificado
    plt.title(title)


In [ ]:
# Aqui nesta linha podemos definir qual o arquivo que desejamos aumentar a qualidade
arquivo = "babuino.png"

# Realiza o processamento necessário para que o python consiga trabalhar com a imagem
imagem = preprocess_image(arquivo)

In [ ]:
# A imagem é transformada em uma matriz de valores que representam os valores RGB de cada PIXEL
imagem

In [ ]:
# Exibe a imagem original
plot_image(imagem, title="Imagem Original")

In [ ]:
# Define o arquivo que possui o modelo de machine learning já treinado para definir os
modelo_h5_path = "modelo_esrgan.h5"

# Realiza a leitura do modelo
modelo = tf.keras.models.load_model(modelo_h5_path, custom_objects={'KerasLayer': hub.KerasLayer})


In [ ]:
# Aplica o modelo treinado na matriz da imagem fornecida e salva o resultado na variável imagem_fake
imagem_fake = modelo(imagem)

In [ ]:
# Exibe a imagem com maior qualidade
plot_image(imagem_fake, title="Qualidade Aumentada")

In [ ]:
# Exibe a imagem original apenas para comparação
plot_image(imagem, title="Imagem Original")

In [ ]:
# Salva a imagem com qualidade aumentada
save_image(imagem_fake, 'babuino Qualidade Aumentada')

E se quisermos montar um código para aumentar a qualidade de diversas imagens?

Temos a pasta "imagens" com algumas imagens de baixa resolução. e iremos salvar as imagens com a qualidade aumentada na pasta "imagens_geradas".


Bem podemos realizar isto com algum loop...

In [ ]:
# Com o comando os.listdir() conseguimos listar todos os arquivos dentro da pasta 'imagens'
os.listdir('imagens')

In [ ]:
# Cria uma lista com o nome das imagens na pasta "imagens"
lista_imagens = os.listdir('imagens')

# Exibe a lista com o nome das imagens
lista_imagens

In [ ]:
lista_imagens = [x for x in lista_imagens if '.ipynb' not in x]
lista_imagens

In [ ]:
for arquivo in lista_imagens:
    # Note que as nossas imagens estão dentro da pasta imagens, 
    # então precisamos definir isto para o python conseguir ler as imagens no local correto
    
    # Realiza o pré processamento da imagem
    imagem = preprocess_image('imagens/'+arquivo)

    # Aplica o modelo treinado na matriz da imagem fornecida e salva o resultado na variável imagem_fake
    imagem_fake = modelo(imagem)

    # Remove a extenção do nome das imagens
    if '.jpeg' in arquivo:
        arquivo = arquivo[:-5]

    elif '.jpg' or '.png' in arquivo:
        arquivo = arquivo[:-4]

    # Salva a imagem com qualidade aumentada
    save_image(imagem_fake, 'imagens_geradas/'+arquivo)